#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [1]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test =  graphlab.SFrame('image_test_data/')

This non-commercial license of GraphLab Create is assigned to luckyskg@gmail.com and will expire on May 23, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-31 15:47:44,009 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\skgoudax\AppData\Local\Temp\graphlab_server_1464689844.log.0


#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [3]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [4]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [5]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [6]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [7]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 376.038ms    |

| Done         |         | 100         | 1.06s        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


We are going to create a simple function to view the nearest neighbors to save typing:

In [8]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [9]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 21.002ms     |

| Done         |         | 100         | 516.052ms    |

+--------------+---------+-------------+--------------+

In [10]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

##Finding similar images to a car

In [11]:
car = image_train[8:9]
car['image'].show()

In [12]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 20.002ms     |

| Done         |         | 100         | 692.069ms    |

+--------------+---------+-------------+--------------+

#Just for fun, let's create a lambda to find and show nearest neighbor images

In [13]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [14]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 23.002ms     |

| Done         |         | 100         | 539.054ms    |

+--------------+---------+-------------+--------------+

In [15]:
show_neighbors(26)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 22.003ms     |

| Done         |         | 100         | 308.031ms    |

+--------------+---------+-------------+--------------+

In [16]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [29]:
dog_train= image_train[image_train['label']=='dog']
cat_train= image_train[image_train['label']=='cat']
auto_train= image_train[image_train['label']=='automobile']
bird_train= image_train[image_train['label']=='bird']

In [18]:
dog_model=graphlab.nearest_neighbors.create(dog_train,
                                          features=['deep_features'],
                                          label='id')

Starting brute force nearest neighbors model training.

In [19]:
cat_model=graphlab.nearest_neighbors.create(cat_train,
                                          features=['deep_features'],
                                          label='id')

Starting brute force nearest neighbors model training.

In [20]:
auto_model=graphlab.nearest_neighbors.create(auto_train,
                                          features=['deep_features'],
                                          label='id')

Starting brute force nearest neighbors model training.

In [21]:
bird_model=graphlab.nearest_neighbors.create(bird_train,
                                          features=['deep_features'],
                                          label='id')

Starting brute force nearest neighbors model training.

In [22]:
get_images_from_ids(cat_model.query(cat_train[0:1])).show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 25.003ms     |

| Done         |         | 100         | 154.015ms    |

+--------------+---------+-------------+--------------+

In [23]:
cat_model.query(cat_train[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 25.003ms     |

| Done         |         | 100         | 159.016ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,33,0.0,1
0,42056,34.2588207732,2
0,41285,34.9140465133,3
0,23963,35.0984380832,4
0,13371,35.1173649827,5


In [24]:
dog_model.query(cat_train[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 25.003ms     |

| Done         |         | 100         | 111.012ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,5755,30.103012871,1
0,16976,31.0805563783,2
0,37685,33.521422,3
0,13387,34.4688257806,4
0,32001,34.6935374921,5


In [25]:
cat_model.query(image_test[0:1]).filter_by([1,2,3,4,5],'rank')['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 14.002ms     |

| Done         |         | 100         | 287.029ms    |

+--------------+---------+-------------+--------------+

36.15573070978294

In [26]:
dog_model.query(image_test[0:1]).filter_by([1,2,3,4,5],'rank')['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 31.003ms     |

| Done         |         | 100         | 311.031ms    |

+--------------+---------+-------------+--------------+

37.77071136184156

In [28]:
dog_test= image_test[image_test['label']=='dog']
cat_test= image_test[image_test['label']=='cat']
auto_test= image_test[image_test['label']=='automobile']
bird_test= image_test[image_test['label']=='bird']

In [30]:
dog_cat_neighbors = cat_model.query(dog_test, k=1)


Starting blockwise querying.

max rows per data block: 7668

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 912.092ms    |

| Done         | 509000  | 100         | 931.093ms    |

+--------------+---------+-------------+--------------+

In [32]:
get_images_from_ids(dog_cat_neighbors)

id,image,label,deep_features,image_array
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
331,Height: 32 Width: 32,cat,"[0.0, 0.0,0.510963916779, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
367,Height: 32 Width: 32,cat,"[1.38658058643, 0.0, 0.0,0.0, 0.0, 0.182891070 ...","[168.0, 151.0, 143.0,145.0, 130.0, 124.0, ..."
494,Height: 32 Width: 32,cat,"[0.0, 0.0539512038231,1.95745122433, 0.0, 0.0, ...","[26.0, 34.0, 29.0, 24.0,29.0, 25.0, 33.0, 43.0, ..."
597,Height: 32 Width: 32,cat,"[0.0, 0.0,0.0470637083054, 0.0, ...","[133.0, 153.0, 138.0,126.0, 146.0, 136.0, ..."
882,Height: 32 Width: 32,cat,"[0.0, 0.0,0.156200289726, 0.0, ...","[141.0, 133.0, 112.0,143.0, 133.0, 113.0, ..."
1252,Height: 32 Width: 32,cat,"[1.57850754261, 0.0, 0.0,0.0, 0.0, 0.0, ...","[217.0, 209.0, 207.0,216.0, 208.0, 205.0, ..."
1696,Height: 32 Width: 32,cat,"[0.0, 0.0,0.109984338284, 0.0, ...","[12.0, 12.0, 12.0, 18.0,18.0, 18.0, 58.0, 58.0, ..."
1846,Height: 32 Width: 32,cat,"[0.0, 0.00756752490997,0.0, 0.523695707321, ...","[60.0, 69.0, 42.0, 54.0,66.0, 38.0, 71.0, 99.0, ..."


In [33]:
knn_model_dog = graphlab.nearest_neighbors.create(dog_train,features=['deep_features'],
                                             label='id')
knn_model_cat = graphlab.nearest_neighbors.create(cat_train,features=['deep_features'],
                                             label='id')
knn_model_car = graphlab.nearest_neighbors.create(auto_train,features=['deep_features'],
                                             label='id')
knn_model_bird = graphlab.nearest_neighbors.create(bird_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [37]:
show_cat_neighbors= lambda i: get_images_from_ids(knn_model_cat.query(image_test[i:i+1]))['image'].show()

In [38]:
show_cat_neighbors(0)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 22ms         |

| Done         |         | 100         | 289ms        |

+--------------+---------+-------------+--------------+

In [40]:
show_neighbors_dogs = lambda i: get_images_from_ids(knn_model_dog.query(image_test[i:i+1]))['image'].show()

In [41]:
show_neighbors_dogs(0)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 26.002ms     |

| Done         |         | 100         | 277.027ms    |

+--------------+---------+-------------+--------------+